# MDH Service Account Token Exchange

This notebook describes how to exchange a Service Account's bearer token for AWS credentials that can be used in other notebooks in this repository.


In [ ]:
# One time setup. Uncomment the following lines if these libraries have not been installed in your python kernel/virtual environment.

#!pip install --upgrade jwt requests cryptography python-dotenv

## Generate a Service Account Key

A [service account](https://developer.mydatahelps.org/api/service_account.html) for your organization is required to issue credentials. Once you have a private key, add it to a `.env` file next to this notebook.

In [ ]:
import jwt
import requests
import pathlib
import os
from datetime import datetime
from dotenv import load_dotenv
from uuid import uuid4

load_dotenv()

In [ ]:
# Derived from https://github.com/CareEvolution/mydatahelps-rest-api-python-quickstart/blob/main/quickstart.py

def get_token() -> str:
    private_key = os.getenv("RKS_PRIVATE_KEY")
    service_account_name = os.getenv("RKS_SERVICE_ACCOUNT")
    token_url = "https://designer.mydatahelps.org/identityserver/connect/token" 

    assertion = {
      "iss": service_account_name,
      "sub": service_account_name,
      "aud": token_url,
      "exp": datetime.now().timestamp() + 900,
      "jti": str(uuid4()),
    }
    signed_assertion = jwt.encode(payload=assertion, key=private_key, algorithm="RS256")
    token_payload = {
      "scope": "api",
      "grant_type": "client_credentials",
      "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
      "client_assertion": signed_assertion
    }
    response = requests.post(url=token_url, data=token_payload)
    response.raise_for_status()
    
    return response.json()["access_token"]

service_account_token = get_token()

## Exchange The Token

With a valid service account token, you can request explorer credentials for your project from the explorer API.

In [ ]:
def exchange_token(token:str) -> dict[str, str]:
    project_code = os.getenv('RKS_PROJECT_CODE')
    api_url = "https://api.explorer.mydatahelps.org/token"

    headers = {
        "Authorization": token,
        "ProjectCode": project_code,
        "Accept": "application/json",
        "Content-Type": "application/json; charset=utf-8",
    }
    response = requests.post(url=api_url, headers=headers)
    response.raise_for_status()

    return response.json()

explorer_credentials = exchange_token(service_account_token)

## Store a Profile

It's easiest to store these credentials in an AWS credentials file for later use.

In [ ]:
def store_credentials(credentials: dict[str, str]) -> None:
    profile_name = os.getenv("AWS_PROFILE_NAME")
    credentials_file = pathlib.Path("~/.aws/credentials").expanduser()

    file_contents = credentials_file.read_text()
    if f"[{profile_name}]" in file_contents:
        # Remove the current profile entry
        split_contents = file_contents.split(f"[{profile_name}]", 1)
        previous_entries = split_contents[0]
        next_entries = split_contents[1].split("\n[", 1)
        file_contents = previous_entries.strip()
        if len(next_entries) > 1:
            file_contents += "\n\n[" + next_entries[1].strip()

    file_contents += f"\n\n[{profile_name}]\n"
    file_contents += f"aws_access_key_id={credentials['AccessKeyId']}\n"
    file_contents += f"aws_secret_access_key={credentials['SecretAccessKey']}\n"
    file_contents += f"aws_session_token={credentials['SessionToken']}\n"

    credentials_file.write_text(file_contents)

store_credentials(explorer_credentials)